In [1]:
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

C:\Users\alexa\AppData\Local\Temp\ipykernel_6240\927923875.py:7: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\alexa\AppData\Local\Temp\ipykernel_6240\927923875.py:8: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
vendas_df = pd.read_csv('Vendas.csv', sep = ';')
produtos_df = pd.read_csv('Produtos.csv', sep = ';')
lojas_df = pd.read_csv('Lojas.csv', sep =';')
clientes_df = pd.read_csv('Clientes.csv', sep = ';')
vendas_df = vendas_df.drop(columns=['ID Canal', 'ID Promocao', 'Data do Envio','Quantidade Devolvida'])

In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load your data
vendas_df = pd.read_csv('Vendas.csv', sep=';')
lojas_df = pd.read_csv('Lojas.csv', sep=';')
clientes_df = pd.read_csv('Clientes.csv', sep=';')
vendas_df = vendas_df.drop(columns=['ID Canal', 'ID Promocao', 'Data do Envio','Quantidade Devolvida'])

# Convert 'Data da Venda' to datetime format and extract the month
vendas_df['Data da Venda'] = pd.to_datetime(vendas_df['Data da Venda'])
vendas_df['Month'] = vendas_df['Data da Venda'].dt.month

# Group by month and calculate total sales
monthly_sales = vendas_df.groupby('Month')['Quantidade Vendida'].sum().reset_index()

# Group by store and calculate total revenue
store_revenue = vendas_df.groupby('ID Loja')['Quantidade Vendida'].sum().reset_index()

# Merge with store names and select top 10 stores
store_revenue = store_revenue.merge(lojas_df, left_on='ID Loja', right_on='ID Loja')
store_revenue = store_revenue.sort_values('Quantidade Vendida', ascending=False).head(10)

# Group by gender and calculate total purchases
gender_purchases = vendas_df.merge(clientes_df, left_on='ID Cliente', right_on='ID Cliente')
gender_purchases = gender_purchases.groupby('Genero')['Quantidade Vendida'].sum().reset_index()

# Mapping from month number to Portuguese name
month_map = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
    5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
    9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}
monthly_sales['Month'] = monthly_sales['Month'].map(month_map)

# Create a Dash application
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    dcc.Graph(id='graph1'),
    dcc.Graph(id='graph2'),
    dcc.Graph(id='graph3'),
])

# Define callback to update graphs
@app.callback(
    [Output('graph1', 'figure'),
     Output('graph2', 'figure'),
     Output('graph3', 'figure')],
    [Input("graph1", "id"),
     Input("graph2", "id"),
     Input("graph3", "id")]
)
def update_figure(id1, id2, id3):
    fig1 = px.bar(monthly_sales, x='Month', y='Quantidade Vendida')
    fig2 = px.bar(store_revenue, x='Nome da Loja', y='Quantidade Vendida')
    fig3 = px.pie(gender_purchases, values='Quantidade Vendida', names='Genero')
    return fig1, fig2, fig3

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\alexa\AppData\Local\Temp\ipykernel_6240\3706922556.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  vendas_df['Data da Venda'] = pd.to_datetime(vendas_df['Data da Venda'])
